## Python Crawler to scrap linkedin recent posts/activity of a user

In [1]:
### Importing required libraries
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs4
import re

In [2]:
### browser params for selenium
firefox_options = Options()
firefox_options.add_argument("--incognito")
firefox_options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe' ## give firefox exe path here


In [3]:
### running the webdriver
driver = webdriver.Firefox(options=firefox_options, executable_path=r"..\driver\geckodriver.exe") ## path where driver is present

C:\Users\Yusra Shahid\AppData\Local\Temp;C\conda_tmp\ipykernel_24512\501550139.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(options=firefox_options, executable_path=r"..\driver\geckodriver.exe") ## path where driver is present


In [4]:
import time
url = "https://linkedin.com/"
driver.get(url)
time.sleep(2)

In [5]:
### sign in to linkedin
signInButton = driver.find_element(By.XPATH,"/html/body/main/section[1]/div/div/form/button")
signInButton.click()

In [6]:
USERNAME = input("Enter the username: ")
PASSWORD = input("Enter the password: ")
print(USERNAME)


Enter the username: Sheikh.hsk@gmail.com
Enter the password: halasexiesyolo12
Sheikh.hsk@gmail.com


In [7]:
email = driver.find_element(By.XPATH,'//*[@id="session_key"]')
email.send_keys(USERNAME)

password = driver.find_element(By.XPATH,'//*[@id="session_password"]')
password.send_keys(PASSWORD)

In [8]:
login = driver.find_element(By.XPATH,'/html/body/main/section[1]/div/div/form/button')
login.click()

In [9]:
### in case there is a phone verification setup page this will skip it
try:
    skip = driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/section/div[2]/div/article/footer/div/div/button[2]')
    skip.click()
except:
    print("skip button not found")

In [11]:
### goto profile and then recent activity link
##profile = driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/div/div[1]/div[1]/a/div[2]')
profile = driver.find_element(By.CSS_SELECTOR,'div.t-16:nth-child(2)')
profile.click()

In [12]:
activity = driver.find_elements(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]/div[3]/div/div/a')
for my_href in activity:
    activity_link = my_href.get_attribute("href")

In [13]:
driver.get(activity_link)
time.sleep(5)

#### getting posts that are gathered in 20 seconds of scroll

In [14]:
start=time.time()
n =20
lastHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight
    end=time.time()
    if round(end-start)>n:
        break

In [15]:
#posts = driver.find_elements(By.CLASS_NAME,"scaffold-finite-scroll__content")
posts_source = driver.page_source 
linkedin_soup = bs4(posts_source.encode("utf-8"), "html")
linkedin_soup.prettify()
containers = linkedin_soup.findAll("div",{"class":"ember-view occludable-update"})

In [16]:
p_text,urls =[],[]

In [17]:
for container in containers:

    try:
        text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper"})
        text = text_box.find("span",{"dir":"ltr"})
        post_text = text.text.strip()
        p_text.append(post_text)
        #print(post_text)
        if "https" in post_text:
            post_url = re.search("(?P<url>https?://[^\s]+)", post_text).group("url")
        else:
            post_url = ""
        #print(post_url)
        urls.append(post_url)
    except:
        #print(text_box)
        pass



In [18]:
print("total number of posts are: ",len(p_text))
print("total number of urls are: ",len(urls)-urls.count(""))

total number of posts are:  4
total number of urls are:  1


In [19]:
p_text

["Another tip for people interested in jobs outside Pakistan:We are generally good developers but what we lack is problem-solving techniques, especially the patterns that can be used to resolve a certain kind of problem.In Pakistan, we mostly work on start-ups or projects which at max last for one year, but this has a very big downside, we don't learn how to evolve the product as it grows.To bridge this gap prepare for Architecture Design Interviews that will help you see the bigger problems before you even start your project.#techinterview #interviewtips #softwaredevelopment #gethired",
 "Celebrating one of our top performer! We are proud to reward Wasi Tariq with the keys to his brand new car as a token of appreciation for his hard work and dedication for the company. Keep driving success! 💯Congratulations on the new set of wheels, now let's see if you can handle the horsepower. 🔥#Folio3 #EmployeeAppreciation #MotivationMonday #CompanyCulture",
 'Since June 2022, we have kept a close

In [20]:
urls

['', '', 'https://lnkd.in/dsYQ75Cx[2]', '']

In [21]:
### goto profile and then recent activity link
profile = driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/div/div[1]/div[1]/a/div[2]')
## profile = driver.find_element(By.CSS_SELECTOR,'div.t-16:nth-child(2)')
profile.click()